<a href="https://colab.research.google.com/github/zalosh12/pyspark_exercises/blob/master/exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ex2 - Getting and Knowing your Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

spark = SparkSession.builder \
    .appName("MyApp") \
    .getOrCreate()


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv).

In [ ]:
!wget -O chipotle.csv https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv


--2025-12-01 13:50:36--  https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 364975 (356K) [text/plain]
Saving to: ‘chipotle.csv’

chipotle.csv        100%[===================>] 356.42K  --.-KB/s    in 0.02s   

2025-12-01 13:50:36 (15.2 MB/s) - ‘chipotle.csv’ saved [364975/364975]



### Step 3. Assign it to a variable called chipo.

In [ ]:
chipo = spark.read.csv('chipotle.csv',header=True,sep="\t", inferSchema=True)

### Step 4. See the first 10 entries

In [ ]:
chipo.show(20)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
|       5|       1| Chips and Guacamole|           

In [ ]:
chipo.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)



### Step 5. What is the number of observations in the dataset?

In [ ]:
# Solution 1
chipo.count()


4622

In [ ]:
# Solution 2
chipo.count()



4622

### Step 6. What is the number of columns in the dataset?

In [ ]:
len(chipo.columns)

5

### Step 7. Print the name of all the columns.

In [ ]:
chipo.columns

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

### Step 8. How is the dataset indexed?

### Step 9. Which was the most-ordered item?

In [ ]:
from pyspark.sql import functions as F

### Step 10. For the most-ordered item, how many items were ordered?

In [ ]:
%%time
most_ordered = (
    chipo.groupBy(F.col('item_name'))
    .agg(
    F.count('*').alias("num_orders"),
    F.sum('quantity').alias("total_quantity")
    )
    .orderBy(F.desc('total_quantity'))
    .limit(1)
    .collect()[0][2]
)

most_ordered

CPU times: user 7.09 ms, sys: 363 µs, total: 7.45 ms
Wall time: 323 ms


761

In [ ]:
%%time
most_ordered = chipo.select(F.mode('item_name')).collect()[0][0]
most_ordered

CPU times: user 4.78 ms, sys: 727 µs, total: 5.51 ms
Wall time: 317 ms


'Chicken Bowl'

### Step 11. What was the most ordered item in the choice_description column?

In [ ]:
# exploded = chipo.select(F.explode("choice_description").alias("item"))
# most_ordered = exploded.select(F.mode('choice_description')).collect()[0][0]
# most_ordered
cleaned = chipo.filter(
    (F.col("choice_description").isNotNull()) &
    (F.col("choice_description") != "") &
    (F.col("choice_description") != "NULL")
)

c = (
    cleaned.groupBy(F.col('choice_description'))
    .agg(
        F.sum('quantity').alias('res'))
    .orderBy(F.desc('res'))
    .limit(1)
    )

c.show()

+------------------+---+
|choice_description|res|
+------------------+---+
|       [Diet Coke]|159|
+------------------+---+



In [ ]:
chipo.select("choice_description").distinct().show(50, truncate=False)


+------------------------------------------------------------------------------------------------------------------------+
|choice_description                                                                                                      |
+------------------------------------------------------------------------------------------------------------------------+
|[Fresh Tomato Salsa, [Fajita Vegetables, Rice, Black Beans, Cheese, Sour Cream]]                                        |
|[[Fresh Tomato Salsa (Mild), Roasted Chili Corn Salsa (Medium)], [Black Beans, Cheese, Guacamole]]                      |
|[Fresh Tomato Salsa, [Fajita Vegetables, Black Beans, Guacamole]]                                                       |
|[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sour Cream, Lettuce]]                                                        |
|[Tomatillo-Red Chili Salsa (Hot), [Rice, Black Beans, Cheese, Lettuce]]                                                 |
|[Tomatillo Gree

### Step 12. How many items were orderd in total?

In [ ]:
chipo.select(F.sum('quantity')).collect()[0][0]

4972

### Step 13. Turn the item price into a float

In [ ]:
%%time

chipo = chipo.withColumn(
    'item_price_float',
    F.regexp_replace('item_price','[$]', '')
    .cast(FloatType())
    )

chipo.show()



+--------+--------+--------------------+--------------------+----------+----------------+
|order_id|quantity|           item_name|  choice_description|item_price|item_price_float|
+--------+--------+--------------------+--------------------+----------+----------------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |            2.39|
|       1|       1|                Izze|        [Clementine]|    $3.39 |            3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |            3.39|
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |            2.39|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |           16.98|
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |           10.98|
|       3|       1|       Side of Chips|                NULL|    $1.69 |            1.69|
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |           11.75|
|       4|

In [ ]:
%%time

chipo = chipo.withColumn(
    'item_price_float',
    F.substring('item_price',2, 1000)
    .cast(FloatType())
    )

chipo.show()

+--------+--------+--------------------+--------------------+----------+----------------+
|order_id|quantity|           item_name|  choice_description|item_price|item_price_float|
+--------+--------+--------------------+--------------------+----------+----------------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |            2.39|
|       1|       1|                Izze|        [Clementine]|    $3.39 |            3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |            3.39|
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |            2.39|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |           16.98|
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |           10.98|
|       3|       1|       Side of Chips|                NULL|    $1.69 |            1.69|
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |           11.75|
|       4|

#### Step 13.a. Check the item price type

In [ ]:
print(chipo.schema['item_price'].dataType)

StringType()


#### Step 13.b. Create a lambda function and change the type of item price

In [ ]:
total = (
    chipo.select(F.sum(F.col('quantity') * F.col('item_price_float')))
    .alias('total_revenue')
    )

total.collect()[0][0]

39237.0197327137

#### Step 13.c. Check the item price type

In [ ]:
print(chipo.schema['item_price'].dataType)

StringType()


### Step 14. How much was the revenue for the period in the dataset?

### Step 15. How many orders were made in the period?

In [ ]:
chipo.select(F.col('order_id')).distinct().count()

1834

### Step 16. What is the average revenue amount per order?

In [ ]:
# Solution 1

chipo = chipo.withColumn('revenue',F.col('item_price_float') * F.col('quantity'))
res = (
    chipo.groupBy('order_id').agg(
        F.sum('revenue').alias('revenue')
    )
)

res.select(F.avg('revenue')).collect()[0][0]



21.394231042919138

In [ ]:
# Solution 2
res.createOrReplaceTempView("res")
result = spark.sql("SELECT AVG(revenue) FROM res")
result.show()


+------------------+
|      avg(revenue)|
+------------------+
|21.394231042919138|
+------------------+



### Step 17. How many different items are sold?

In [ ]:
chipo.select('item_name').distinct().count()

50